In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import matplotlib as plt
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import random
import numpy as np

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [ ]:
#
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
class VAE(nn.Module):
  def __init__(self, x_dim,h_dim1, h_dim2, z_dim):
    super(VAE, self).__init__()

    #creating the required NNS for the ENCODER part
    self.fc1=nn.Linear(x_dim,h_dim1)
    self.fc2=nn.Linear(h_dim1,h_dim2)
    self.fc01=nn.Linear(h_dim2,z_dim) # for the mean
    self.fc02=nn.Linear(h_dim2,z_dim) # for the varianve

    #now for he decoder part'''
    self.fc3=nn.Linear(z_dim,h_dim2)
    self.fc4=nn.Linear(h_dim2,h_dim1)
    self.fc5=nn.Linear(h_dim1,x_dim)

  def encoder(self,x):
    h=f.relu(self.fc1(x))
    h=f.relu(self.fc2(h))
    m=self.fc01(h) # computed the mean tensor
    v=self.fc02(h) # computed the log of the std dev
    return m, v

  def sampling(self,m,v):
    std = torch.exp(0.5*v)
    eps = torch.randn_like(std)
    return eps.mul(std).add(m) # returns the sampled space from the available latent space

  def decoder(self,z):
    h=f.relu(self.fc3(z))
    h=f.relu(self.fc4(h))
    return f.sigmoid(self.fc5(h)) # here we could also experiment with the tanh function int=stead of sigmoid

  def forward(self,x):
    m, lv= self.encoder(x.view(-1,784))
    z= self.sampling(m,lv)
    return self.decoder(z), m, lv


#building an instance of the class or called the model xD
vae=VAE(x_dim=784, h_dim1=512,h_dim2= 256, z_dim=2)

#setting up the device diagnostic code here also
if torch.cuda.is_available():
  vae.cuda()

In [ ]:
vae


VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc01): Linear(in_features=256, out_features=2, bias=True)
  (fc02): Linear(in_features=256, out_features=2, bias=True)
  (fc3): Linear(in_features=2, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=784, bias=True)
)

In [ ]:
optimizer = optim.Adam(vae.parameters(), lr=0.01)

def loss_function(recon_x, x, mu, log_var):
    BCE = f.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    #MSE = f.mse_loss(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var-2*(np.log(2)) - ((log_var.exp()+((mu-1).pow(2)))/2)  )
    return BCE + KLD
    #return MSE + KLD

In [ ]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 546.926211
Train Epoch: 1 [10000/60000 (17%)]	Loss: 201.799961
Train Epoch: 1 [20000/60000 (33%)]	Loss: 178.966074
Train Epoch: 1 [30000/60000 (50%)]	Loss: 175.975449
Train Epoch: 1 [40000/60000 (67%)]	Loss: 175.985410
Train Epoch: 1 [50000/60000 (83%)]	Loss: 164.948906
====> Epoch: 1 Average loss: 183.5451
====> Test set loss: 167.8566
Train Epoch: 2 [0/60000 (0%)]	Loss: 178.364824
Train Epoch: 2 [10000/60000 (17%)]	Loss: 168.202734
Train Epoch: 2 [20000/60000 (33%)]	Loss: 173.209570
Train Epoch: 2 [30000/60000 (50%)]	Loss: 169.133340
Train Epoch: 2 [40000/60000 (67%)]	Loss: 166.037656
Train Epoch: 2 [50000/60000 (83%)]	Loss: 155.354912
====> Epoch: 2 Average loss: 165.5620
====> Test set loss: 162.8011
Train Epoch: 3 [0/60000 (0%)]	Loss: 163.841504
Train Epoch: 3 [10000/60000 (17%)]	Loss: 157.629424
Train Epoch: 3 [20000/60000 (33%)]	Loss: 155.305840
Train Epoch: 3 [30000/60000 (50%)]	Loss: 172.035820
Train Epoch: 3 [40000/60000 (67%)]	Loss: 165.12

In [ ]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()

    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')